# Technical Factor - Parabolic SAR (SAR)

In [1]:
import dai
import pandas as pd

In [2]:
sd = '2025-01-01'
ed = '2026-01-01'

instrument_list = ['600519.SH']

In [3]:
sql = """
WITH
data_base AS (
    SELECT
        date,
        instrument,
        open,
        high,
        low,
        close,
        volume,
    FROM cn_stock_bar1d
),

data_factor AS (
    SELECT
        date,
        instrument,
        close,
        m_ta_sar(high, low) AS SAR,
    FROM data_base
),

data_signal_trend AS (
    SELECT
        date,
        instrument,
        IF(close > SAR, 1, 0) AS TRBY1,
        IF(close < SAR, 1, 0) AS TRSL1,
    FROM data_factor
),

data_signal_momentum AS (
    SELECT
        date,
        instrument,
        IF(close > SAR AND (close - m_lag(close, 1)) > 0, 1, 0) AS MTBY1,
        IF(close < SAR AND (close - m_lag(close, 1)) < 0, 1, 0) AS MTSL1,
    FROM data_factor
),

data_signal_reversal AS (
    SELECT
        date,
        instrument,
        IF(m_lag(close, 1) <= m_lag(SAR, 1) AND close > SAR, 1, 0) AS RVBY1,
        IF(m_lag(close, 1) >= m_lag(SAR, 1) AND close < SAR, 1, 0) AS RVSL1,
    FROM data_factor
),

data_signal_breakout AS (
    SELECT
        date,
        instrument,
        IF(m_lag(close, 1) <= m_lag(SAR, 1) AND close > SAR AND (close - m_lag(close, 1)) > 0, 1, 0) AS BKBY1,
        IF(m_lag(close, 1) >= m_lag(SAR, 1) AND close < SAR AND (close - m_lag(close, 1)) < 0, 1, 0) AS BKSL1,
    FROM data_factor
),

data_combined AS (
    SELECT
        date,
        instrument,
        close,
        SAR,
        TRBY1,
        TRSL1,
        MTBY1,
        MTSL1,
        RVBY1,
        RVSL1,
        BKBY1,
        BKSL1,
    FROM data_factor
    JOIN data_signal_trend USING (date, instrument)
    JOIN data_signal_momentum USING (date, instrument)
    JOIN data_signal_reversal USING (date, instrument)
    JOIN data_signal_breakout USING (date, instrument)
)

SELECT *
FROM data_combined
QUALIFY COLUMNS(*) IS NOT NULL
ORDER BY date, instrument
"""

In [4]:
df = dai.query(sql, filters={"date":[sd,ed], "instrument":instrument_list}).df()
df

,date,instrument,close,SAR,TRBY1,TRSL1,MTBY1,MTSL1,RVBY1,RVSL1,BKBY1,BKSL1
0,2025-01-03,600519.SH,12015.218095,12418.359209,0,1,0,1,0,0,0,0
1,2025-01-06,600519.SH,11730.111225,12408.994670,0,1,0,1,0,0,0,0
2,2025-01-07,600519.SH,11731.740407,12379.493310,0,1,0,0,0,0,0,0
3,2025-01-08,600519.SH,11750.476001,12351.172005,0,1,0,0,0,0,0,0
4,2025-01-09,600519.SH,11762.694867,12307.388371,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
237,2025-12-25,600519.SH,11945.482630,11624.554267,1,0,1,0,0,0,0,0
238,2025-12-26,600519.SH,11945.144750,11646.501181,1,0,0,0,0,0,0,0
239,2025-12-29,600519.SH,11842.682738,11667.131279,1,0,0,0,0,0,0,0
240,2025-12-30,600519.SH,11738.953676,11686.523572,1,0,0,0,0,0,0,0
